 <font size="6">Сверточные нейронные сети</font>

# Введение в сверточные нейронные сети


## Полносвязная нейронная сеть 
Fully-connected Neural Network (FCN). В современных статьях чаще используется термин Multilayer Perceptron (MLP).

На прошлом занятии мы рассмотрели следующий подход для классификации изображений:

1. Превращаем исходные данные в вектор. 

>***Примечание***: при обработке цветного изображения размером $32\times32$ пикселя ($32\times32\times3$), размерность входного вектора будет равна $3072$. Однако, в общем случае, модель получает на вход набор одноразмерных векторов (матрицу), потому при обработке одного изображения размер матрицы будет $3072\times1$.

2. Перемножаем матрицу данных с матрицей весов. Размер последней может быть, например, $100\times3072$. Где $3072$ - размер входного вектора, а $100$ - количество признаков, которое мы хотим получить. Результат обработки одного изображения будет иметь размер $100\times1$.

3. Поэлементно применяем к полученной матрице нелинейную функцию (функцию активации), например Sigmoid или ReLU. Размерность данных при этом не меняется ($100\times1$). В результате получаем вектор активаций или признаков.

4. Используем полученные признаки как входные данные для нового слоя. Количество весов слоя будет зависеть от размерности входной матрицы и того, что мы хотим получить на выходе. Если мы делаем классификатор на $10$ классов, то матрица весов должна иметь размерность $10\times100$, и на этом можно остановиться. Но в общем случае количество слоев может быть произвольным.


<img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/fully_connected_neural_network.png" width="600">


На изображении представлена описанная выше нейронная сеть, функцией активации в которой является ReLU. Добавление второго слоя позволило модели использовать более одного шаблона на класс. Можно убедиться в этом, обучив модель на датасете CIFAR-10 и визуализировав веса первого слоя модели.

<img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/mlp_templates.jpg" width="700">

За счёт создания нескольких шаблонов для каждого из классов, многослойные архитектуры в общем случае показывают более высокую, чем перцептроны, эффективность на задачах классификации изображений. Однако подход с использованием многослойного перцептрона также имеет свои недостатки. Чтобы обнаружить их, присмотримся к происходящему с данными при предобработке.

## Нарушение связей между соседними пикселями

Единственным этапом предобработки данных при использовании многослойного перцептрона является превращение изображения в вектор. Основной его проблемой является "потеря связи" между соседними пикселями, что показано на изображении ниже. В общем случае, значения из соседних пикселей могут оказаться на большом расстоянии внутри результирующего вектора. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/img_to_vector_problem.png" width="600">

Словом, после преобразования, мы теряем информацию о взаимном расположении значений на исходном изображении - при подборе весов никак не используется информация о том, что какие-то значения были \"близки\" друг к другу на исходном изображении. При этом мы, люди, понимаем важность взаимосвязей между пикселями

К примеру, набор тёмных пикселей находящихся поблизости внутри изображения может сообщить нам, что там располагается некий тёмный объект. Если же мы преобразуем изображение в вектор и эти значения "разбросает" по нему без сохранения пространственной структуры, то новому наблюдателю будет очень сложно понять, что где-то на исходном изображении была тёмная зона.  



## Примеры 'hand-crafted' фильтров

Для реализации идеи локальности при обработке данных используется операция свёртки. Свёртка требует тензор параметров - \"ядро свёртки\", также называемое фильтром. 

В течение долгого времени веса в фильтрах подбирались вручную. Этого было достаточно для простой обработки изображений. О примерах мы поговорим далее.

### Фильтр Гаусса

Данный фильтр является одним из наиболее известных и используется в компьютерном зрении для снижения уровня шума в изображении. Такое название он носит, поскольку веса внутри фильтра инициируются двумерной функцией Гаусса. Центр распределения находится в центре фильтра. 

Ниже показан пример использования фильтра Гаусса размером $5 \times 5$ для размытия изображения. Размер фильтров может варьировать от задачи к задаче - то есть, в некоторых случаях использование фильтра $5 \times 5$ даёт слишком сильное размытие, из-за чего может быть использован фильтр меньшего размера: к примеру, $4 \times 4$ или $3 \times 3$.

In [ ]:
import torch
from torch import nn
from skimage import data
import random
import numpy as np
import matplotlib.pyplot as plt
from skimage.util import random_noise

# fix random_seed
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

img_cat = data.cat().mean(axis = 2).astype('int32')
img_cat_noised = random_noise(img_cat / 255, mode='gaussian') * 255

g_kernel5 = torch.tensor([[1,4,7,4,1],
                          [4,16,26,16,4],
                          [7,26,41,26,7],
                          [4,16,26,16,4],
                          [1,4,7,4,1]]
                          ,dtype=torch.float) / 273 # sum of weights

gaussian_filter = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=g_kernel5.shape, bias=False)
gaussian_filter.weight.data = g_kernel5.unsqueeze(0).unsqueeze(0) # gaussian_kernel shape: [1,1,5,5]

input = torch.FloatTensor(img_cat_noised).unsqueeze(0).unsqueeze(0) # image to torch tensor shape: [1,1,300,451]
out = gaussian_filter(input)
out = out.squeeze(0).squeeze(0)

fig, axes = plt.subplots(1, 2, figsize=(20,5))
axes[0].imshow(img_cat_noised, cmap='gray', vmin=0, vmax=255)
# use .detach() to detached from the current graph(Requires grad = False) 
axes[1].imshow(out.detach().numpy(), cmap='gray', vmin=0, vmax=255)
axes[0].set(title=f'Noised image, shape: {img_cat_noised.shape}')
axes[1].set(title=f'Blurred image: {out.detach().numpy().shape}')
axes[0].axis('off')
axes[1].axis('off')
plt.show()

### Фильтр Собеля

На заре компьютерного зрения, людям приходилось выполнять feature engeneering при обработке изображений в явном виде. То есть, было необходимо извлекать важную информацию из изображений. Довольно важной информацией на изображении являются границы - они могут быть использованы для большого количества задач (например, для поиска дорожной разметки на изображении). Как видно на изображении ниже, фильтры - в частности, фильтр Собеля - могут прекрасно справляться с поиском границ. С этим фильтром Вы ещё раз столкнётесь в ходе практической работы.

Пусть $A$ — это исходное изображение, а $G_{x}$ и $G _{y}$ — два изображения, на которых каждая точка содержит приближённые производные по $x$ и по $y$. Они вычисляются следующим образом:

$G_y=\begin{bmatrix}+1&+2&+1\\0&0&0\\-1&-2&-1\\\end{bmatrix}*A \\ G_x=\begin{bmatrix}-1&0&+1\\-2&0&+2\\-1&0&+1\\\end{bmatrix} *A $

где $*$ обозначает двумерную операцию свертки.

Координата $x$ здесь возрастает «направо», а $y$ — «вверх». В каждой точке изображения приближённое значение величины градиента можно вычислить путём использования полученных приближенных значений производных:

$G= \sqrt{{G_x^2}+{G_y^2}}$ (имеется в виду поэлементно).

Используя эту информацию, мы можем также вычислить направление градиента:

$\Theta = arctan(\frac{G_y}{G_x})$,

где, к примеру, угол Θ равен нулю для вертикальной границы, у которой тёмная сторона слева

In [ ]:
img_cat = data.cat().mean(axis = 2).astype('int32')

class Sobel(nn.Module):
    def __init__(self):
        super().__init__()
        self.filter = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3, stride=1, padding=0, bias=False)

        Gx = torch.tensor([[-1.0, 0.0, 1.0], 
                           [-2.0, 0.0, 2.0],
                           [-1.0, 0.0, 1.0]])
        
        Gy = torch.tensor([[1.0, 2.0, 1.0], 
                           [0.0, 0.0, 0.0],
                           [-1.0, -2.0, -1.0]])

        G = torch.cat([Gx.unsqueeze(0), Gy.unsqueeze(0)], 0)
        G = G.unsqueeze(1)
        self.filter.weight = nn.Parameter(G, requires_grad=False)

    def forward(self, img):
        x = self.filter(img)
        x = torch.mul(x, x)
        x = torch.sum(x, dim=1, keepdim=True)
        x = torch.sqrt(x)
        return x

sobel_filter = Sobel()
input = torch.FloatTensor(img_cat).unsqueeze(0).unsqueeze(0) # image to torch tensor shape: [1,1,300,451]
out = sobel_filter(input)
out = out.squeeze(0).squeeze(0)

fig, axes = plt.subplots(1, 2, figsize=(20,5))
axes[0].imshow(img_cat, cmap='gray', vmin=0, vmax=255)
# use .detach() to detached from the current graph(Requires grad = False) 
axes[1].imshow(out.detach().numpy(), cmap='gray', vmin=0, vmax=255)
axes[0].set(title=f'Original image, shape: {img_cat.shape}')
axes[1].set(title=f'After sobel filter image, shape: {out.detach().numpy().shape}')
axes[0].axis('off')
axes[1].axis('off')
plt.show()

#### Детектор границ Canny

На самом деле, если мы будем применять упомянутый выше алгоритм без подготовки изображения и дополнительных операций, то не получим хорошее качество обнаружения границ. Поэтому существуют детекторы границ, дополняющие оператор Собеля, либо иные операторы (в алгоритме ниже оператор Собеля используется на шаге 2).

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L06/edge_detection_canny.png" width="850">

#### Детекторы углов

Другим примером созданных вручную экстракторов признаков являются всевозможные детекторы углов. Одним из самых известных среди них является детектор углов Харриса. Мы не будем на нем останавливаться, но Вы можете прочесть о нем самостоятельно в заметке [Corner Detector](https://en.wikipedia.org/wiki/Harris_Corner_Detector).


Как Вы увидите далее, сверточные нейронные сети по факту автоматизируют процесс извлечения подобных признаков, высвобождая время исследователей и инженеров.

## Свертка с фильтром

По сути своей, операция свёртки - это та же самая взвешенная сумма с добавлением свободного члена, используемая в полносвязных линейных слоях. 

Ключевое отличие между линейным слоем и свёрткой заключается в том, что линейный слой получает на вход всё изображение, тогда как свёртка - небольшие фрагменты изображения. То есть, если при обработке линейным слоем всё изображение сравнивалось с шаблонами, то теперь лишь небольшие фрагменты изображения сравниваются с шаблонами.

К примеру, на GIF ниже можно увидеть, как фильтр размером $3\times3$ применяется к одноканальному изображению размером $5\times5$. Шаблон имеет форму x-образного креста. В правой части можно увидеть, насколько фрагмент изображения под фильтром совпадает с шаблоном внутри фильтра.

<img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/convolution_with_filter.gif" width="300">

Реализуем это в коде:

In [ ]:
img = torch.Tensor([[1,1,1,0,0],
                    [0,1,1,1,0],
                    [0,0,1,1,1],
                    [0,0,1,1,0],
                    [0,1,1,0,0]])

kernel = torch.Tensor([[1, 0, 1],
                       [0, 1, 0],
                       [1, 0, 1]])

result = torch.zeros((3,3)) # img - kernel + 1 (5 - 3 + 1 = 3)

for i in range(0, result.shape[0]):
    for j in range(0, result.shape[1]):
        segment = img[i:i+kernel.shape[0], 
                        j:j+kernel.shape[1]]
        result[i, j] = torch.sum(segment * kernel)

print(f'img shape: {img.shape}')
print(f'kernel shape: {kernel.shape}')
print(f'result shape: {result.shape}')
print(f'result:\n{result}')

# Сверточный слой нейросети

## Основные параметры свёртки

Как уже упоминалось ранее, по сути, операция свёртки представляет из себя вычисление взвешенной суммы со свободным членом, что довольно сильно напоминает линейный слой с тем лишь отличием, что последний применяется ко всем данным, а не к их части. Тем не менее, подобно линейному слою, операцию свёртки можно встроить в нейросеть и, путём градиентного спуска, подбирать параметры свёртки. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/img_and_convolution_filter.png" width="300">

Реализуем операцию свертки с помощью линейного слоя:

In [ ]:
import torch
import torch.nn as nn

local_linear = nn.Linear(9, 1, bias = False) # 9 = 3*3 (weights shape: (3,3))

img = torch.Tensor([[1,1,1,0,0],
                    [0,1,1,1,0],
                    [0,0,1,1,1],
                    [0,0,1,1,0],
                    [0,1,1,0,0]])
# kernel weights
weights = torch.Tensor([[1, 0, 1],
                       [0, 1, 0],
                       [1, 0, 1]])

local_linear.weight = nn.Parameter(weights.reshape(-1)) # set weights 

result = torch.zeros((3,3)) # img - kernel + 1 (5 - 3 + 1 = 3)

for i in range(0, result.shape[0]):
    for j in range(0, result.shape[1]):
        segment = img[i:i+weights.shape[0], 
                        j:j+weights.shape[1]].reshape(-1)
        result[i, j] = local_linear(segment)

print(f'img shape: {img.shape}')
print(f'weights shape: {weights.shape}')
print(f'result shape: {result.shape}')
print(f'result:\n{result}')

### Обработка цветных/многоканальных изображений

В примерах выше мы рассматривали [черно-белые](https://ru.wikipedia.org/wiki/%D0%A7%D1%91%D1%80%D0%BD%D0%BE-%D0%B1%D0%B5%D0%BB%D0%B0%D1%8F_%D1%84%D0%BE%D1%82%D0%BE%D0%B3%D1%80%D0%B0%D1%84%D0%B8%D1%8F) изображения. Их также называют одноканальными изображениями, т.к. в них цвет пикселя определяется одним числом, характеризующим яркость.

В работе мы будем часто сталкиваться с трехканальными цветными [RGB](https://ru.wikipedia.org/wiki/RGB) изображениями, в которых цвет пикселя определяется тремя числами, характеризующими три основных цвета (красный, зеленый и синий). 

Анализируемое изображение может иметь и большее количество каналов. Например: марсоход Opportunity для получения изображений использовал [13 фильтров](https://habr.com/ru/post/160621/).  

В общем случае, изображение имеет размер $C \times H\times W$. К примеру, изображения из датасета CIFAR-10 имеют размер $3\times32\times32$.  

При обработке подобных изображений, используется уже не матрица, а трёхмерный тензор. Ниже демонстрируется процесс обработки фрагмента трёхмерного изображения фильтром размерности $3\times3$:

<img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/rgb_image_and_convolution_filter.png" width="400">


Результатом свертки входного тензора с одним фильтром будет карта признаков с глубиной 1, вне зависимости от количества каналов входного тензора.

Во время прямого прохода фильтр перемещается по ширине и высоте входного тензора и вычисляются скалярные произведения между значениями фильтра и соответствующими значениями входного тензора. Так формируется двумерная карта признаков, которая содержит результат применения данного фильтра к каждой из областей входного тензора.

В качестве входного тензора можно использовать не изображение, а результат предыдущего сверточного слоя. 


<img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/convolution_filter_forward_pass.png" width="200">

In [ ]:
img = torch.randn((3, 8, 8)) # 3-num of channels, (8,8)-img size
kernel = torch.randn((3, 3, 3)) # 3-num of filters, (3,3)-kernel size

result = torch.zeros(6, 6) # 8 - 3 + 1 = 6

for i in range(result.shape[0]):
  for j in range(result.shape[1]):
    segment = img[:, 
                    i:i+kernel.shape[0], 
                    j:j+kernel.shape[1]]
    result[i, j] = torch.sum(segment * kernel)

print(f'img shape: {img.shape}')
print(f'kernel shape: {kernel.shape}')
print(f'result shape: {result.shape}')

### Использование нескольких фильтров

Подобно тому, как в перцептроне использовались несколько выходных нейронов, чтобы передать информацию о сходстве входного вектора с различными шаблонами, можно использовать несколько фильтров при свёртке. То есть, выполнять свёртку несколько раз ($C_{out}$, по количеству фильтров)  с разными фильтрами. В результате появится несколько карт признаков, имеющих размер $1\times H_{out}\times W_{out}$. При объединении этих карт будет получен тензор размерности $C_{out}\times H_{out} \times W_{out}$, где $C_{out}$ - количество фильтров, а также количество "каналов" в полученном представлении. Полученное представление может быть без проблем передано для следующей операции свёртки. 

На изображении ниже продемонстрирован результат применения сверточного слоя, содержащего $5$ фильтров, к изображению из CIFAR-10. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/convolution_layer_with_5_filters.png" width="400"> 

В свёрточном слое указаны параметры, необходимые для создания ядра свёртки - количество каналов во входном представлении `in_channels` и размер ядра свёртки `kernel_size`. Также в нём необходимо указывать количество используемых фильтров `out_channels`. Стоит отметить, что, в отличие от полносвязанного слоя, свёрточный слой не требует информации о количестве значений во входном представлении и может быть использован как для представлений $C_{in} \times 32 \times 32$, так и $C_{in} \times 100 \times 100$. Словом, представления могут иметь практически любой размер, главное чтобы пространственные размеры не были меньше размеров ядра свёртки.


In [ ]:
conv = torch.nn.Conv2d(in_channels=3,  # Number of input channels (3 for RGB images)
                       out_channels=5, # Number of filters/output channels
                       kernel_size=3)

img = torch.randn((1,3,100,100)) # 1-batch size, 3-num of channels, (100,100)-img size
print(f'img shape: {img.shape}')

out = conv(img)
print(f'out shape: {out.shape}') # [1, 5, 98, 98]

### Уменьшение размера представлений

Заметим, что при свёртке ширина $W_{out}$ и высота $H_{out}$ **карты признаков** будут отличаться от **пространственных размерностей** $W_{in}$ и $H_{in}$ исходного тензора. К примеру, при обработке трёхканального тензора размера $32 \times 32$ ядром размера $5 \times 5$, можно будет произвести лишь $27$ сдвигов $(32 - 5)$ по вертикали и столько же - по горизонтали. Но при этом размер итоговой карты признаков будет равен $28 \times 28$, поскольку первый ряд (либо столбец) можно получить без сдвигов по вертикали либо горизонтали, соответственно. При повторном применении фильтра, размер каждой из сторон уменьшится ещё на $4$. Итоговое значение $N'$ пространственной размерности $N$ при размере стороны фильтра $F$ вычисляется по следующей формуле: $N' = N - F + 1$.

<img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/decrease_size_of_image_after_convolution.png" width="700">

In [ ]:
conv_1 = torch.nn.Conv2d(in_channels=3, # Number of input channels (3 for RGB images)
                       out_channels=6, # Number of filters/output channels
                       kernel_size=5)

conv_2 = torch.nn.Conv2d(in_channels=6, # Number of input channels (3 for RGB images)
                              out_channels=10, # Number of filters/output channels
                              kernel_size=5)

img = torch.randn((1,3,32,32)) # 1-batch size, 3-num of channels, (32,32)-img size
print(f'img shape: {img.shape}')

out_1 = conv_1(img)
print(f'out_1 shape: {out_1.shape}') # [1, 6, 28, 28]

out_2 = conv_2(out_1)
print(f'out_2 shape: {out_2.shape}') # [1, 10, 24, 24]

Заметим, что при уменьшении размера представлений пиксели, находящиеся около краёв, участвуют в значительно меньшем количестве свёрток, чем пиксели в середине, хотя информация в них не обязательно менее ценна, чем информация из центральных пикселей. К примеру, пиксель в верхнем левом углу представления вне зависимости от размера фильтра будет принимать участие лишь в одной свёртке и информация о нём будет сохранена лишь в верхнем левом углу нового представления. 

### Расширение (padding)  

Для борьбы с описанной выше проблемой применяется *набивка/дополнение* входного тензора (англ. *padding*). В ходе него ширина и высота тензора увеличиваются за счёт приписывания столбцов и строк с некими значениями. К примеру, на изображении ниже перед свёрткой ядром размера $3\times3$ был применён padding нулями.  

<img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/padding.png" width="250">

На примере убедимся, что это позволит нам сохранить пространственные размерности тензоров.

In [ ]:
img = torch.randn((1, 1, 5, 5)) # create random image
print(f'Original tensor:\nshape:{img.shape}:\n {img}')

# add zeros to image manually
padded_img = torch.zeros((1, 1, 7, 7)) # create zeros array to insert image in center
padded_img[:, :, 1:-1, 1:-1] += img # insert image, we get image arounded by zeros
print(f'\nPadded tensor:\nshape:{padded_img.shape}:\n {padded_img}')

# define two conv layers to check results
conv_3 = torch.nn.Conv2d(in_channels=1, 
                        out_channels=1, 
                        kernel_size=3)
conv_5 = torch.nn.Conv2d(in_channels=1, 
                        out_channels=1, 
                        kernel_size=5)
# use layers separately, to compare results
conved_3 = conv_3(img)
conved_5 = conv_5(img)

conved_pad_3 = conv_3(padded_img)
conved_pad_5 = conv_5(padded_img)

print('\n\nOriginal shape:', img.shape)
print('Shape after convolution layer(kernel 3x3):', conved_3.shape)
print('Shape after convolution layer(kernel 5x5):', conved_5.shape)

print('\n\nPadded shape:', padded_img.shape)
print('Shape after convolution with padding(kernel 3x3):', conved_pad_3.shape)
print('Shape after convolution with padding(kernel 5x5):', conved_pad_5.shape)

Заметим, что дополнение одним рядом и одним столбцом не является универсальным решением: для фильтра размером 5 размер выходного тензора всё равно отличается от входного. Если мы немного видоизменим полученную выше формулу (используя размер дополнения $P$), то получим следующую формулу: $N' = N + 2\cdot P - F + 1$.  
Видно, для того чтобы пространственные размеры не изменялись ($N' = N$), для разных размеров фильтра требуются разные размеры паддинга. В общем случае, для размера фильтра $F$, требуемый размер дополнения  $\displaystyle P = \frac{F-1}{2}$. 


Важными терминами, связанными с паддингом, являются "*same padding*" - размер выходного тензора не отличается от размера входного тензора, а также "*valid padding*" - отсутствие паддинга.

```
"VALID" = без паддинга:

   inputs:         1  2  3  4  5  6  7  8  9  10 11 (12 13)
                  |________________|                dropped
                                 |_________________|
"SAME" = с паддингом нулями:

               pad|                                      |pad
   inputs:      0 |1  2  3  4  5  6  7  8  9  10 11 12 13|0  0
               |________________|
                              |_________________|
                                             |________________|
```

Ниже можно увидеть пример обработки RGB изображения с same padding (с использованием 0):

<img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/convolution_with_same_padding_rgb_image.gif" width="780">

Теперь реализуем padding, используя инструментарий torch и сравним его с ручным добавлением padding:

In [ ]:
img = torch.randn((1, 1, 5, 5)) # define random image

# add zeros manually
padded_img = torch.zeros((1, 1, 7, 7)) 
padded_img[:, :, 1:-1, 1:-1] += img

# conv layer without padding (padding=0 by default)
conv_3 = torch.nn.Conv2d(in_channels=1, 
                        out_channels=1, 
                        kernel_size=3,
                        padding = 0)

# conv layer with padding = 1 (add zeros)
conv_3_padded = torch.nn.Conv2d(in_channels=1, 
                                out_channels=1, 
                                kernel_size=3,
                                padding = 1) # Padding added 1 zeros line to all four sides of the input
original = conv_3(padded_img)
padded = conv_3_padded(img)

print(f'Explicitly padded:\n {original.shape}\n {original}')
print(f'\nImplicitly padded:\n{padded.shape}\n  {padded}')
print("\nExplicitly padded shape = Implicitly padded shape")

## Использование свёрточных слоёв

### Свёрточный слой = свёртка + активация

Поскольку операция свертки является линейной, аналогично полносвязному слою, к выходу сверточного слоя применяется* функция активации (например, ReLU).

>*Так как функция активации применяется к тензору поэлементно, не важно, какую именно форму имеет тензор, а значит и какой слой находился перед ней: полносвязанный или сверточный. 

In [ ]:
input = torch.randn((1,3,32,32))

model = torch.nn.Sequential(
                           nn.Conv2d(in_channels = 3,
                                     out_channels = 6,
                                     kernel_size = 3), # after conv shape: [1,6,30,30]
                           nn.ReLU(), # Activation doesn't depend on input shape
                           nn.Flatten(), # 6*30*30 = 5400
                           nn.Linear(5400,100), 
                           nn.ReLU(),  # Activation doesn't depend on input shape
                           nn.Linear(100,10) # 10 classes, like a cifar10
                           )

out = model(input)
print(f'out shape: {out.shape}')
print(f'out:\n{out}')

>Отметим, что внутри нейросети появляется новый слой - `nn.Flatten()`. Он необходим, поскольку полносвязанный слой принимает на вход набор векторов значений, тогда как свёрточный слой возвращает набор трёхмерных тензоров. `nn.Flatten()` необходим, чтобы преобразовать набор тензоров в набор векторов. Эту же операцию можно было бы выполнить с помощью метода `.view(-1)`.

### Общая структура свёрточной нейронной сети

Нейросетевая модель из предыдущего примера позволяет в общем случае понять структуру свёрточных нейронных сетей: после некоторого количества свёрточных слоёв, извлекающих локальную пространственную информацию, идут полносвязанные слои (как минимум в количестве одного), сопоставляющие извлечённую информацию. 

Внутри свёрточных слоёв происходит следующий процесс: первые слои нейронных сетей имеют малые рецептивные поля, т.е. им соответствует малая площадь на исходном изображении. Такие нейроны могут активироваться лишь на некие простые шаблоны, по типу углов или освещённости.
 Активации уровнем выше уже имеют большие рецептивные поля, в результате чего в картах активации может сохраняться информация о более нетривиальных геометрических фигурах. С каждым слоем свёрточной нейронной сети рецептивные поля увеличиваются, увеличивается и сложность шаблонов, на которые может реагировать слой. В некоторых случаях, рецептивное поле нейронов может быть размером со всё исходное изображение. Пример можно увидеть на схеме ниже.

<img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/receptive_field_size.png" width="700">

Если изначально рецептивное поле имело размер $1\times1$, то после свёртки фильтром $K\times K$, оно стало иметь размер $K \times K$. То есть, рецептивное поле каждого нейрона увеличилось на $K-1$ по каждому из направлений. Не сложно самостоятельно убедиться, что данная закономерность сохранится при дальнейшем применении фильтров того или иного размера.

Однако, при больших размерах изображений (к примеру, $1024\times1024$), нейронная сеть становится очень глубокой, что приводит к появлению большого количества параметров. Тем не менее, рецептивное поле каждого из нейронов растить нужно, что делается путём искусственного увеличения рецептивного поля каждого из элементов тензора. В принципе, это можно сделать двумя принципиально разными способами, о примере каждого из которых мы и поговорим:  
1. Доработать операцию свёртки, чтобы рецептивное поле росло быстрее
2. Добавить некую промежуточную операцию, которая бы увеличивала рецептивные поля

### Шаг свёртки (Stride)

Вариантом доработки свёртки с целью увеличения рецептивного поля нейрона является изменение шага фильтра. Ранее фильтры двигались с шагом 1 по горизонтали и 1 по вертикали, то есть зоны применения фильтров были сдвинуты друг относительно друга на 1 строку либо столбец. При увеличении шага, количество применений фильтра изменяется. 

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/convolution_parameter_stride.gif" width="350"></center>
<center>Свёртка массива $5\times5$ фильтром размером $3\times3$ с шагом $2$ по вертикали и горизонтали.</center>

Казалось бы, с увеличением шага $S$ рецептивное поле не выросло - как увеличивалось с $1$ до $K$, так и увеличивается. Однако обратим внимание на иное: если раньше размерность $N$ становилась $N - F + 1$, то теперь она станет $\displaystyle 1 + \frac{N-F}{S}$. В результате, если раньше следующий фильтр с размером $K'$ имел рецептивное поле в $\displaystyle N \cdot \frac{K'}{N'} = N \cdot \frac{K'}{N - F + 1}$, то теперь $\displaystyle N \cdot \frac{K'}{N'} = N \cdot \frac{K'}{1 + \frac{N-F}{S}}$. Понятно, что $\displaystyle \frac{K'}{N - F + 1} \leq \frac{K'}{1 + \frac{N-F}{S}}$, потому рецептивное поле каждого нейрона увеличивается.

При этом важно заметить, что в некоторых случаях часть данных может не попасть в свёртку. К примеру, при $N = 7,\, K = 3,\, S = 3$. В данном случае, $\displaystyle N' = 1 + \frac{7 - 3}{3} = 2\frac13.$ В подобных ситуациях часть изображения не захватывается, в чём мы можем убедиться на наглядном примере:

In [ ]:
# Create torch tensor 7x7
input = torch.tensor([[[[1,1,1,1,1,1,99],
                        [1,1,1,1,1,1,99],
                        [1,1,1,1,1,1,99],
                        [1,1,1,1,1,1,99],
                        [1,1,1,1,1,1,99],
                        [1,1,1,1,1,1,99],
                        [1,1,1,1,1,1,99]]]],dtype=torch.float)

print(f'input shape: {input.shape}')

conv = torch.nn.Conv2d(in_channels=1, # Number of channels
                       out_channels=1, # Number of filters
                       kernel_size=3, 
                       stride = 3,
                       bias = False # Don't use bias
                       )
conv.weight  = torch.nn.Parameter(torch.ones((1,1,3,3))) # Replace randow weights to ones
out = conv(input) 

print(f'out shape: {out.shape}')
print(f'out:\n{out}')

### Уплотнение (Субдискретизация, Pooling)

Другим вариантом стремительного увеличения размера рецептивного поля является использование дополнительных слоёв, требующих меньшее количество вычислительных ресурсов. Слои субдискретизации прекрасно выполняют эту функцию - подобно свёртке, производится разбиение изображения на небольшие сегменты, внутри которых выполняются операции, не требующие использования обучаемых весов. Два популярных примера подобных операций: получение максимального значения (max pooling) и получение среднего значения (average pooling). 


Аналогично разбиению на сегменты при свёртке, слои пуллинга имеют два параметра: размер фильтра $F$ (то есть, каждого из сегментов) и шаг $S$ (stride). Аналогично свёрткам, при применении пуллинга, формула размера стороны будет $\displaystyle N' = 1+ \frac{N-F}{S}.$

Ниже приведён пример использования обоих пуллингов при обработке массива.

<img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/subdiscretization_pooling.png" width="600">

Реализуем это в коде:

In [ ]:
# create tensor 4x4
input = torch.tensor([[[
                     [1,1,2,4],
                     [5,6,7,8],
                     [3,2,1,0],
                     [1,2,3,4],
                     ]]],dtype=torch.float)

max_pool = torch.nn.MaxPool2d(kernel_size = 2, stride=2)
avg_pool = torch.nn.AvgPool2d(kernel_size = 2, stride=2)

print("Input:\n",input)
print("Max pooling:\n",max_pool(input))
print("Average pooling:\n",avg_pool(input))

**Важно отметить**, что, в отличие от свёрток, уплотнение выполняется по каждому из каналов отдельно, в результате чего количество каналов не меняется, в отличие от применения фильтра при свёртке. К примеру, ниже можно увидеть визуализация применения max pooling к одному из каналов тензора, имеющего $64$ канала. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/changing_size_of_image_after_pooling.png" width="350">

### Свёртка фильтром $1\times1$

Оба упомянутых выше метода позволяют значительно снизить количество весов в модели, путём быстрого увеличения рецептивного поля нейронов, что даёт возможность не делать модель очень глубокой. Однако если мы посмотрим на количество весов в слое нейронной сети, то заметим, что проблема может быть не только в глубине модели. 

Вспомним, что по сути своей при свёртке значения сегмента пропускаются через перцептрон с $C_{out}$ выходами и $C_{in} \cdot F_1 \cdot F_2$ входами, где $F_1$ и $F_2$ обозначают размер фильтра модели. Итого оказывается, что модель имеет $C_{out} \cdot C_{in} \cdot F_1 \cdot F_2$ весов и $C_{out}$ свободных членов, что в сумме даёт нам $C_{out} \cdot (C_{in} \cdot F_1 \cdot F_2 + 1)$ параметров. Каждая из четырёх переменных оказывает значительное влияние на количество весов. Уменьшение $F_1$ и $F_2$ может привести к уменьшению рецептивных полей в свёрточных слоях выше, что повлечёт увеличение количества слоёв и, скорее всего, увеличение общей сложности модели. Уменьшение $C_{out}$ может привести к ухудшению обобщающей способности моделей. В связи с этим всем, был придуман трюк для уменьшения $C_{in}$ - свёртка фильтром $1\times1$.

Идея заключается в следующем: для каждого элемента в карте активаций $H_{in}\times W_{in}$ есть вектор размерностью $C_{in}$. Элементы этого вектора сообщают, насколько сильно рецептивное поле соответствует каждому из $C_{in}$ шаблонов. Возможно, стоит попробовать найти некие полезные комбинации данных "признаков" внутри каждого элемента представления. Для этого прекрасно подходит поэлементно применённый перцептрон с $C_{temp}$ выходами, также являющийся набором из $C_{temp}$ фильтров свёртки размером $C_{in}\times1\times1$. Ниже приведён пример применения такого фильтра с целью снижения количества каналов.

<img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/convolution_with_kernel_size_one.png" width="500">

In [ ]:
conv = torch.nn.Conv2d(in_channels=64, # Number of input channels
                       out_channels=32, # Number of filters
                       kernel_size=1)

input = torch.randn((1,64,56,56))
out = conv(input)

print("Input shape:",input.shape)
print("Shape after 1x1 conv:",out.shape) # [1, 32, 56, 56] batch, C_out, H_out, W_out

Теперь давайте выведем формулу, которая покажет, насколько использование промежуточного свёрточного слоя с $C_{temp}$ слоями уменьшит количество параметров при свёртке с $C_{out}$ фильтрами $F_1\times F_2$.

**Количество параметров при наивном решении:**  
$Param_{naive} = C_{out} \cdot (C_{in} \cdot F_1 \cdot F_2 + 1) = C_{in} \cdot (C_{out} \cdot F_1 \cdot F_2) + C_{out}.$

**Количество параметров при решении с $1\times1$:**  
$Param_{1\times1} = C_{temp} \cdot (C_{in}\cdot1\cdot1 + 1) + C_{out} \cdot (C_{temp} \cdot F_1 \cdot F_2 + 1) = C_{temp} \cdot (1 + C_{in} + C_{out}\cdot F_1 \cdot F_2) + C_{out}.$

Не сложно заметить, что $\displaystyle \frac{Param_{1\times1}}{Param_{naive}} \approx \frac{C_{temp}}{C_{in}}.$

### Сравнение свёрточного и полносвязанного слоев 


<img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/convolution_layer.png" width="550">

Мы успели довольно подробно рассмотреть свёрточный слой и заметить большое количество его сходств с полносвязным слоем (перцептроном). Теперь давайте попробуем сравнить эти два слоя по требуемому количеству параметров и операций умножения.  

В первую очередь, давайте определимся с размерами входного и получаемого тензоров. Пусть на вход передаётся $C_{in}\times H_{in}\times W_{in}$. На выходе пусть будет $K$ нейронов для полносвязного слоя и $C_{out}\times H_{out} \times W_{out}$ для свёрточного слоя (фильтр имеет размер $C_{in} \times F_1 \times F_2$). $F_1$ и $F_2$ - размер ядра свёртки по высоте и ширине, соответственно.

Для простоты расчётов давайте примем, что шаг фильтра равен $1$ как по горизонтали, так и по вертикали. В таком случае, $H_{out} = H_{in} - F_1 + 1$, а $W_{out} = W_{in} - F_2 + 1$.
  
##### Количество параметров:  
***Полносвязанный слой:***  
Данный слой требует по параметру (весу) для всех связей между входными и выходными нейронами, то есть $C_{in} \cdot H_{in} \cdot W_{in} \cdot K$. Помимо этого, каждый из выходных нейронов имеет свободный член, общее количество которых $K$. Итого, количество обучаемых параметров: $(C_{in} \cdot H_{in} \cdot W_{in} + 1) \cdot K$.

***Свёрточный слой:***  
Для свёрточного слоя параметры связаны лишь с ядрами свёртки - внутри каждого ядра находятся $C_{in} \cdot F_1 \cdot F_2$ параметров, общее их количество - $C_{out}$. Помимо этого, каждое из ядер имеет свой собственный свободный член, потому общее количество обучаемых параметров: $(C_{in} \cdot F_1 \cdot F_2 + 1) \cdot C_{out}$.

***Сравнение количества параметров:***  
Поскольку количество свободных членов мало относительно количества весов, мы опустим их в этих расчётах. 
$$Comp_{param} = \frac{C_{in} \cdot H_{in} \cdot W_{in} \cdot K}{C_{in} \cdot F_1 \cdot F_2 \cdot C_{out}} = \frac{H_{in} \cdot W_{in} \cdot K}{F_1 \cdot F_2 \cdot C_{out}}.$$  


##### Количество умножений: 
***Полносвязанный слой:***  
В данном слое каждый вес используется лишь один раз, в результате общее количество умножений - $C_{in} \cdot H_{in} \cdot W_{in} \cdot K$.  

***Свёрточный слой:***
Заметим, что в отличие от перцептрона, свёрточная нейронная сеть использует каждый вес несколько раз, а именно - при подсчёте каждого из элементов карты активации. Размер карты активаций: $H_{out} \times W_{out}$. В итоге оказывается, что операция умножения применяется $C_{in} \cdot F_1 \cdot F_2 \cdot C_{out} \cdot H_{out} \cdot W_{out}$.

***Сравнение количества умножений:***
$$Comp_{mult} = \frac{C_{in} \cdot H_{in} \cdot W_{in} \cdot K}{C_{in} \cdot F_1 \cdot F_2 \cdot C_{out} \cdot H_{out} \cdot W_{out}} = \frac{H_{in} \cdot W_{in} \cdot K}{F_1 \cdot F_2 \cdot C_{out} \cdot H_{out} \cdot W_{out}} = \frac{Comp_{param}}{H_{out} \cdot W_{out}}.$$

##### Выводы:

Заметим, что выигрыш по количеству параметров при использовании свёрточного слоя омрачается большим количеством операций перемножения. Это было проблемой в течение долгого времени, пока вычисление операции свёртки не перевели на видеокарты (Graphical Processing Unit). При выполнении свёртки одного сегмента не требуется информация о результатах свёртки на другом сегменте, потому чисто теоретически можно выполнять данные операции параллельно, с чем как раз прекрасно справляются видеокарты. 

#### Другие виды сверток


##### 1D

Свертка - это математическая операция, которая объединяет две функции в третью. Например, допустим, что у нас есть две функции: $f(t)$ и $g(t)$. Мы хотим вычислить площадь их пересечения: $f(t) \times g (t) = (f \times g) (t)$



В этом случае мы применяем **г(т)**(называется ядром) более **е(т)** и изменение ответа **(Е* г) (т)** согласно пересечению над областью обеих функций. Эта концепция свертки является наиболее используемой техникой обработки сигналов, поэтому она также применяется в компьютерном зрении, что можно рассматривать как обработку сигнала от нескольких датчиков RGB. Интуиция в эффективности свертки заключается в их способности фильтровать заданные входные сигналы в объединенный, более полезный результат.

В конкретном случае изображений сигнал лучше понимается с точки зрения матриц, а не форм волны. Поэтому наши функции **(т)а** также **г(т)** теперь станет изображение *(матрица)* а также Ядро *(матрица)* соответственно. 
Еще раз, мы можем применить свертку, сдвинув одну из матриц поверх другой:

<img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/1d_convolution.gif" width="700">



$$\large(f*g)(x)\stackrel{def}{=}  \int\limits_{-\infty}^{\infty}  f(y)g(x-y)dy$$




То, что вы видите выше, это просто матричное умножение скользящего окна на изображении с ядром, а затем сложение суммы. Сила сверток в контексте Computer Vision заключается в том, что они являются отличными экстракторами функций для области датчиков RGB. При индивидуальном рассмотрении каждый пиксель (датчик RGB) бессмысленно понимать, что содержит изображение. Именно отношения друг с другом в пространстве придают образу истинный смысл. Это относится к тому, как вы читаете эту статью на своем компьютере, когда пиксели, представляющие символы, и ваш мозг сопоставляют черные пиксели друг с другом в пространстве, чтобы сформировать концепцию символов.

[torch.nn.Conv1d](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html) 

```
torch.nn.Conv1d(in_channels,
                out_channels, 
                kernel_size, 
                stride=1, 
                padding=0, 
                dilation=1,
                groups=1, 
                bias=True, 
                padding_mode='zeros')
```

In [ ]:
conv = nn.Conv1d(16, 33, 3, stride=2)
input = torch.randn(20, 16, 50)
output = conv(input)
print(output.shape)

##### 3D

Одномерная операция свёртки используется для данных, имеющих последовательную структуру - текстов, аудиозаписей, цифровых сигналов. Как правило, данную структуру можно представить в виде временного измерения.  
Двумерная операция свёртки, о которой мы сегодня много говорили, применяется для обработки данных, имеющих пространственную структуру - то есть, играют роль взаимные расположения как по вертикали, так и по горизонтали. (добавить пример не изображения, но каналы + пространственная размерность + время!)  
Трёхмерная версия свёртки используется, когда данные имеют три независимых "пространственных" компоненты. Простейшим примером являются видео: к двумерной структуре самих изображений добавляется координата времени.

<img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/3d_convolution.png" width="600">





```
 torch.nn.Conv3d(in_channels, 
                  out_channels, 
                  kernel_size, 
                  stride=1, 
                  padding=0, 
                  dilation=1, 
                  groups=1, 
                  bias=True, 
                  padding_mode='zeros')
 
```



In [ ]:
# With cubic kernels and same stride
conv = nn.Conv3d(in_channels = 16, 
                 out_channels = 33, 
                 kernel_size = 3, 
                 stride=2)

# non-square kernels with unequal stride and padding
conv = nn.Conv3d(in_channels = 16, 
                 out_channels = 33, 
                 kernel_size = (3, 5, 2), 
                 stride=(2, 1, 1),
                 padding=(4, 2, 0))

input = torch.randn(20, 16, 10, 50, 100)
out = conv(input)

print('out shape: ',out.shape)

#### Справочник по сверткам

[A guide to convolution arithmetic for deep
learning](https://arxiv.org/pdf/1603.07285v1.pdf) 

[An Introduction to different Types of Convolutions in Deep Learning](https://towardsdatascience.com/types-of-convolutions-in-deep-learning-717013397f4d)


the best: 

[A Comprehensive Introduction to Different Types of Convolutions](https://towardsdatascience.com/a-comprehensive-introduction-to-different-types-of-convolutions-in-deep-learning-669281e58215) 


## Пример сверточной сети

Теперь мы можем более детально взглянуть на типичную архитектуру свёрточной нейронной сети. Как ранее уже обсуждалось, в первую очередь необходимо последовательностью свёрточных слоёв и уплотнений достичь того, чтобы каждый элемент карты активации имел большое рецептивное поле, а значит, мог отвечать за большие и сложные шаблоны. Затем данные карты активаций выпрямляются в вектора и передаются в полносвязанные слои, последовательность которых, используя глобальную информацию, возвращает значение целевой переменной.

<img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/neural_network_architecture.png" width="800">

#### LeNet

<img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/lenet_architecture.png" width="900">

Примером сети построенной по такой архитектуре является LeNet.
Была разработана в 1989г [Яном Ле Куном](https://en.wikipedia.org/wiki/Yann_LeCun). Сеть имела 5 слоев с обучаемыми весами, из них 2 сверточных.

Применялась в США для распознавания рукописных чисел на почтовых конвертах до начала 2000г.

[LeNet torch documentation](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html)

Ниже представлена реализация подобной сети на Pytorch для датасета MNIST:

Загрузим датасет:

In [ ]:
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from IPython.display import clear_output

# transforms for data
transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5), (0.5))])

train_set = MNIST(root='./MNIST', train=True, download=True, transform=transform)
test_set = MNIST(root='./MNIST', train=False, download=True, transform=transform)

batch_size = 64
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

clear_output()

Напишем сверточную сеть:

In [ ]:
class CNN_model(nn.Module):

    def __init__(self):
        super(CNN_model, self).__init__()
        self.conv_stack = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), # in channel=1, out=32
            nn.MaxPool2d(2), # size [32,14,14]
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1), # in channel=32, out=32
            nn.MaxPool2d(2), # size [32,7,7]
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(32*7*7, 100), # in = channel*heght*width
            nn.ReLU(),
            nn.Linear(100, 10)
        )

    def forward(self, x): 
        x = self.conv_stack(x) 
        return x

Запустим обучение:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # change run time to gpu to fast training

model = CNN_model().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
loss_hist = [] # for plotting
for epoch in range(num_epochs):
    hist_loss = 0
    for _, batch in enumerate(train_loader, 0): # get batch
        # parse batch 
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.to(device)
        # sets the gradients of all optimized tensors to zero.
        optimizer.zero_grad() 
        # get outputs
        Y_pred = model(imgs) 
        # calculate loss
        loss = criterion(Y_pred, labels)
        # calculate gradients
        loss.backward() 
        # performs a single optimization step (parameter update)
        optimizer.step()
        hist_loss += loss.item()
    loss_hist.append(hist_loss /len(train_loader))
    print(f"Epoch={epoch} loss={loss_hist[epoch]:.4f}")

Построим график обучения:

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(num_epochs), loss_hist)
plt.xlabel("Epochs",fontsize=15)
plt.ylabel("Loss",fontsize=15)
plt.show()

Давайте посчитаем accuracy:

In [ ]:
def calaculate_accuracy(model, data_loader):
    correct, total = 0, 0 
    with torch.no_grad(): 
        for batch in data_loader: # get batch
            imgs, labels = batch # parse batch
            imgs, labels = imgs.to(device), labels.to(device)
            Y_pred = model.forward(imgs) # get output
            _, predicted = torch.max(Y_pred.data, 1) # get predicted class
            total += labels.size(0) # all examples
            correct += (predicted == labels).sum().item() # correct predictions 
    return correct / total 

In [ ]:
acc_train = round(calaculate_accuracy(model, train_loader), 3)
print(f"Accuracy train = {acc_train}")
acc_test = round(calaculate_accuracy(model, test_loader), 3)
print(f"Accuracy test = {acc_test}")

Если мы сравним результат с моделью, которую мы делали на прошлом занятии, то можем увидеть, как выросла точность и уменьшилась ошибка на обучении (точность выросла на ~10%, ошибка уменьшилась с 0.4 до ~ 0.01)

# Визуализация


Нам может быть интересно, на какую информацию обращает внимание модель в процессе работы, на какие визуальные шаблоны реагирует, насколько они интерпретируемы? 

Чтобы ответить на все эти вопросы, можно визуализировать карты активаций и веса фильтров свёртки.

<img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/what_hidden_layers.png" width="1000">

## Визуализация весов 

Веса фильтров на первом слое легко визуализировать. И результат легко интерпретируется, так как у фильтров такое же количество каналов, как и у цветных изображений (3).

Ниже приведен пример того, как это можно сделать для обученной модели в Pytorch (Alexnet).

Чтобы понять, через какие свойства можно получить доступ к весам, выведем структуры модели

In [ ]:
from torchvision import models
alexnet = models.alexnet(weights='AlexNet_Weights.DEFAULT') 
print(alexnet)

Видно, что первый слой это 0-й элемент контейнера features.
Веса хранятся в weight.data

In [ ]:
weight_tensor = alexnet.features[0].weight.data # extract weights
print('Weights shape',weight_tensor.shape)

Чтобы отобразить все веса на одном изображении воспользуемся вспомогательной функцией [make_grid](https://pytorch.org/vision/stable/utils.html#torchvision.utils.make_grid) из [torchvision.utils](https://pytorch.org/vision/stable/utils.html)

На вход метод получает batch изображений (B x C x H x W) в формате torch.tensor и визуализирует их в виде таблице. 

In [ ]:
from torchvision import utils

img_grid = utils.make_grid((weight_tensor+1)/2,pad_value=1) # combine weights from all channel into table, note remapping to (0,1) range
print("Output is CxHxW image",img_grid.shape)

Ее часто используют, чтобы отображать изображения в tensorboard 

А чтобы отобразить получившуюся таблицу в блокноте средствами matplotlib, нам потребуется поменять порядок хранения данных, поместив каналы на первое место.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["figure.figsize"] = (12,12)
plt.imshow(np.transpose(img_grid, (1, 2, 0))) # change channel order for compability with numpy & matplotlib
plt.show()

Видно, что модель научились улавливать простые геометрические формы - края под разными углами, точки того или иного цвета. Тем не менее, фильтры AlexNet'а оказались настолько большими, что частично захватили не только простую локальную информацию, но и сложные градиенты или решётки.


### Визуализация фильтров промежуточных слоев

К сожалению, выполнить ту же операцию для фильтров на скрытых слоях едва ли представляется разумным - в отличие от принятого трёхканального вида фильтров, который легко можно трактовать и визуализировать, фильтры поздних слоёв имеют гораздо больше каналов, что делает их удобное отображение практически невозможным. Пожалуй, единственным вариантом отображения является поканальное отображение весов, которое довольно сложно трактовать, в чём можно убедиться, взглянув на пример ниже.


**Higher Layer: Visualize Filter**
<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L06/weight_visualization.png" width="700">

<center><p><em>Source: <a href="https://cs.stanford.edu/people/karpathy/convnetjs/demo/cifar10.html">ConvNetJS CIFAR-10 example</a></p> </em></center>

Визуализируем веса 2-го сверточного слоя AlexNet. 
Слой доступен через `features[3] `

In [ ]:
weights_of_conv2_layer = alexnet.features[3].weight.data # extract weights
print(weights_of_conv2_layer.shape)

В нем 192 фильтра в каждом 64 ядра. Поэтому ограничимся первым фильтром и выведем все его ядра.

In [ ]:
first_filter_kernels =  weights_of_conv2_layer[0]
print(first_filter_kernels.shape)

Чтобы использовать image_grid входной тензор должен иметь формат BxCxHxW.  Поэтому добавим размерность соответствующую каналам:

In [ ]:
img_grid = utils.make_grid(
    weights_of_conv2_layer[0].unsqueeze(1), # add fake channel dim
    pad_value=1)

plt.rcParams["figure.figsize"] = (16,16)
plt.imshow(np.transpose((img_grid+1)/2, (1, 2, 0))) # change channel order for compability with numpy
plt.show()

Интерпретация такой визуализации довольно затруднительна, зато мы разобрались, как получать доступ к весам.

## Визуализация карт активаций

Наиболее очевидный метод визуализации заключается в том, чтобы показать активации сети во время прямого прохода. Для сетей ReLU активации обычно начинают выглядеть относительно сгущенными и плотными, но по мере развития обучения активации обычно становятся более редкими и локализованными. 

На последних слоях свёрточной нейронной сети размеры рецептивных полей нейронов становятся сравнимы с размером исходного изображения, потому при визуализации их карт активации становится понятно, какие нейроны реагируют на какие части изображений.

К примеру, на изображении ниже активация выделенного нейрона достигнута благодаря пикселям, примерно соответствующим расположению лица человека, потому можно предположить, что он научился находить лица на изображении. Более подробно об этом можно почитать в статье [Understanding Neural Networks Through Deep Visualization](https://arxiv.org/abs/1506.06579).

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L06/visualization_activations.png" width="700">

В отличие от весов, карты активаций не сохраняются в памяти. Для того чтобы получить к ним доступ, в pytorch предусмотрен механизм под названием [Hooks](https://pytorch.org/tutorials/beginner/former_torchies/nnft_tutorial.html#forward-and-backward-function-hooks)

Благодаря ему можно получить доступ к выходам или входам слоя, как при прямом, так и при обратном распространении сигнала через сеть.



Зарегистрируем свой hook. Он просто выведет в консоль размеры карты активации (выхода слоя).



In [ ]:
from torch import nn
def module_hook(module: nn.Module, input, output):  # For nn.Module objects only.
    print("Hi, i am hook_1!",output.shape) # activation_map

handle = alexnet.features[10].register_forward_hook(module_hook) # attach hook to last conv layer

Проверим, что он работает

In [ ]:
import torch
out = alexnet(torch.randn(1,3,224,224))

Чтобы удалить hook, используйте метод remove дескриптора, который возвращает метод register_forward_hook

In [ ]:
handle.remove()
out = alexnet(torch.randn(1,3,224,224))

Вывода нет, hook отключился!

Теперь напишем hook, который выведет нам карту активации. 
Так как на выходе данного слоя 256 каналов, выведем каждый отдельно, подав на вход make_grid тензор с 256 элементами. 

Для этого потребуется:
* удалить batch измерение
* добавить измерение, имитирующее канал для картинок




In [ ]:
def module_hook(module: nn.Module, input, output):
    #activation_map = output.squeeze(0).unsqueeze(1) # alternative solution
    activation_map = output.permute(1,0,2,3)
    print(activation_map.shape)
    img_grid = utils.make_grid(activation_map,pad_value=10,nrow = 16) 
    plt.rcParams["figure.figsize"] = (12,12)
    plt.imshow(np.transpose(img_grid, (1, 2, 0))) # change channel order for compability with numpy
    plt.show()

handle = alexnet.features[10].register_forward_hook(module_hook)



 

Чтобы карта активаций  была интерпретируема, надо использовать реальное изображение. Загрузим его:

In [ ]:
!wget -q https://edunet.kea.su/repo/EduNet-web_dependencies/L06/fox.jpg

In [ ]:
from PIL import Image
img_fox = Image.open("fox.jpg")

plt.rcParams["figure.figsize"] = (8,8)
plt.imshow(img_fox)
plt.axis('off')
plt.show()

Загрузим изображение, преобразуем в тензор и подадим на вход модели:

In [ ]:
from torchvision import transforms

transform = transforms.Compose([transforms.Resize((256,256)),
                                transforms.ToTensor()])

tensor = transform(img_fox)
out = alexnet(tensor.unsqueeze(0))

Тут уже можно увидеть некоторые паттерны. Видно, что многие фильтры реагируют на лисицу.

Единственная опасная ловушка, которую можно легко заметить с помощью этой визуализации, заключается в том, что некоторые карты активации могут быть равны нулю для многих различных входов. Это может указывать на мертвые фильтры и может быть симптомом высокой скорости обучения.

Отключим наш hook, чтобы он не мешал дальнейшим экспериментам

In [ ]:
handle.remove()

## Feature extractor


Альтернативным способом понимания эффективности обучения сети является анализ карт активаций с последних слоёв нейронной сети. Поскольку в процессе работы нейросети создаются промежуточные представления, имеющие меньшую размерность, весь процесс представляет из себя проектирование данных в пространство меньшей размерности. Новое представление может быть использовано для решения различных задач не только с помощью линейных слоёв нейросети, но и с помощью других методов машинного обучения.

В следующих примерах мы будем использовать представление, полученное на последнем скрытом слое свёрточной нейронной сети.

**Последний слой**

<img src ="https://edunet.kea.su/repo/EduNet-content/L06/img_license/feature_extractor.png" width="700">

Чтобы убедиться в полезности полученных представлений, выполним поиск ближайшего соседа для изображения в исходном виде, пиксельном пространстве. В результате будут получены изображения, не всегда относящиеся к тому же классу, что и исходное изображение.

Если же мы используем предобученную свёрточную нейронную сеть для получения представлений изображений в более низкой размерности, и затем применим алгоритм поиска соседей, то "близко" друг к другу окажутся изображения, принадлежащие одному классу, даже если есть визуально более похожие изображения из иных классов - фон и прочие неважные части изображения имеют меньшее влияние на представление. Успешное извлечение признаков отображает успешное обучение модели. Один из вариантов выполнения кластеризации изображений - кластеризация промежуточных представлений свёрточных нейронных сетей.

**Последний слой: ближайшие соседи**
<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L06/last_layer_knn.jpg" width="900">